# Загрузка данных

Импорт и установка библиотек

In [1]:
# Установка необходимых библиотек
!pip install spacy nltk scikit-learn transformers datasets iterative-stratification wordcloud
!python -m spacy download ru_core_news_md
!pip install evaluate
# Импорт библиотек
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import json
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


# Анализ и предобработка

### просмотр статистической информации

In [2]:
# # === ШАГ 1: Объединяем два исходных файла ===
# df1 = pd.read_csv("/content/project-13-at-2025-05-11-01-19-e254a3e5.csv")
# df2 = pd.read_csv("/content/project-20-at-2025-05-15-05-35-2831ec76.csv")
# df = pd.concat([df1, df2], ignore_index=True)
df = pd.read_csv("/content/321.csv")
# === ШАГ 2: Разметка sentiment ===
known_categories = [
    "Вопрос решен",
    "Нравится качество выполнения заявки",
    "Нравится качество работы сотрудников",
    "Нравится скорость отработки заявок",
    "Понравилось выполнение заявки"
]

# Добавляем колонки (в том числе "Другое")
for col in known_categories + ["Другое"]:
    if col not in df.columns:
        df[col] = 0

df["sentiment"] = df["sentiment"].fillna("")

for i, row in df.iterrows():
    try:
        sentiment_data = json.loads(row["sentiment"])
        choices = sentiment_data.get("choices", [])

        for col in known_categories + ["Другое"]:
            df.at[i, col] = 0  # обнуляем

        for choice in choices:
            if choice in known_categories:
                df.at[i, choice] = 1
            else:
                df.at[i, "Другое"] = 1
    except Exception:
        for col in known_categories + ["Другое"]:
            df.at[i, col] = 0

# Сохраняем объединённый с разметкой файл
df.to_csv("/content/merged_with_sentiment_flags.csv", index=False)

# === ШАГ 3: Сравнение по ID с разметкой ===
df_base = pd.read_csv("/content/разметка комментариев 2.csv")
df_other = pd.read_csv("/content/merged_with_sentiment_flags.csv")

if 'id' in df_base.columns and 'id' in df_other.columns:
    df_matched = df_other[df_other['id'].isin(df_base['id'])]

    # Сбрасываем индекс, чтобы он начинался с 1 (а не с 0)
    df_matched = df_matched.reset_index(drop=True)
    df_matched.index = df_matched.index + 1  # Делаем индексацию с 1

    df_matched.to_csv("/content/коментарии.csv", index=True)  # index=True сохраняет новый индекс
    print("Файл коментарии.csv сохранён успешно.")
else:
    print("В одном из файлов отсутствует колонка 'id'.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/321.csv'

In [ ]:
print("\n=== Информация о DataFrame (с дубликатами) ===")
print(f"Общее количество строк: {len(df_matched)}")
print(f"Количество уникальных строк: {len(df_matched.drop_duplicates())}")
print(f"Количество полных дубликатов: {len(df_matched) - len(df_matched.drop_duplicates())}")

In [ ]:
print("\n \n ===Типы данных и пропуски: ===")
print(df_matched.info())

In [ ]:
df_matched.head(10)

In [ ]:
df_matched.tail(10)

In [ ]:
df_matched = df_matched.dropna()

In [ ]:
df_matched.describe()

In [ ]:
df_matched.shape

In [ ]:
#Удаление колонок не нужных для обучения модели и не применяющиеся в анализе
df_clean =df_matched.drop(columns=['annotation_id', 'annotator', 'id','updated_at', 'lead_time','sentiment', 'created_at'])

In [ ]:
df_clean.info()

In [ ]:
df_clean.head(15)

### Визуализация

In [ ]:
# Маска из изображения здания
mask = np.array(Image.open("/content/building (1).png"))

# Объединяем все комментарии в одну строку
text_all = " ".join(df_clean['comment'].dropna())

# Создание облака слов без стоп-слов
wordcloud = WordCloud(
    width=2000,
    height=1800,
    background_color='white',
    mask=mask,
    contour_color='black',
    contour_width=3,
    max_words=1500,
    colormap='plasma',
    prefer_horizontal=0.95,
    max_font_size=200,
    scale=5,  # увеличивает детализацию!
    random_state=42
).generate(text_all)

# Отображение
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.show()




На этом графике можно уведить что больше всего люди пишут отзывы используя такие слова как спасибо,быстро,оперативно и т.д

In [ ]:
# Явно указываем столбцы для исключения
exclude_cols = ['comment', 'rating']
category_cols = [col for col in df_clean.columns if col not in exclude_cols]

# Строим график
plt.figure(figsize=(10, 6))
df_clean[category_cols].sum().sort_values().plot(
    kind='barh',
    color='red',
    edgecolor='darkblue',  # Добавляем границы для лучшей читаемости
    alpha=0.7  # Полупрозрачность
)

# Улучшаем оформление
plt.title("Количество положительных откликов по категориям", pad=20, fontsize=14)
plt.xlabel("Количество", labelpad=10)
plt.ylabel("Категория", labelpad=10)
plt.grid(True, axis='x', linestyle='--', alpha=0.6)

# Добавляем значения на столбцы
for i, v in enumerate(df_clean[category_cols].sum().sort_values()):
    plt.text(v + 0.5, i, str(v), color='black', va='center')

plt.tight_layout()
plt.show()

На этом графике можно заменить очень сильный дисбаланс классов

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Подсчёт и нормализация
rating_percent = df_clean['rating'].value_counts(normalize=True).sort_index() * 100

# Настройка стиля
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Создаем цветовую палитру
colors = sns.color_palette("viridis", len(rating_percent))

# Построение графика
bars = plt.bar(rating_percent.index.astype(str), rating_percent.values, color=colors)

# Добавляем данные на каждый столбец
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%',
             ha='center', va='bottom',
             fontsize=10)

# Настройка оформления
plt.title("Распределение оценок (rating)\n", fontsize=14, fontweight='bold')
plt.ylabel("Процент (%)", fontsize=12)
plt.xlabel("Оценка", fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)

# Убираем лишние границы
sns.despine(left=True)

# Добавляем горизонтальную сетку
plt.grid(axis='y', alpha=0.3)

# Оптимизация расположения элементов
plt.tight_layout()

plt.show()

На этом графике видно распредление оценок в процентном соотношении и наблюдаеться что больше всего данных это 5 звезд

## Работа с текстом

In [ ]:
MODEL_NAME = "blanchefort/rubert-base-cased-sentiment"  # Поддерживает эмодзи и русский
MAX_LENGTH = 256            # Увеличьте длину для контекста (если позволяет память)
NUM_LABELS = 6
SEED = 42
TRAIN_BATCH_SIZE = 32      # Уменьшите batch для стабильности обучения
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 3e-5        # Повышенный LR для tiny-модели
WEIGHT_DECAY = 0.01
NUM_EPOCHS = 35            # Уменьшите эпохи + ранняя остановка
WARMUP_RATIO = 0.2          # Больше прогревов
LOGGING_STEPS = 100
SAVE_TOTAL_LIMIT = 2
OUTPUT_DIR = "./results"
LOGGING_DIR = "./logs"

# Добавьте раннюю остановку
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(early_stopping_patience=5)

### Очистка текста

In [ ]:
import re
import pandas as pd
from datasets import Dataset
import torch


# Очистка текста, не удаляя пунктуацию
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[\r\n]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()  # Не удаляем пунктуацию, эмодзи и спецсимволы

# Очистка комментариев
df_clean['comment'] = df_clean['comment'].apply(clean_text)

# Категории для классификации
category_cols = [
    'Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Другое'
]

# Подготовка DataFrame
df_model = df_clean[['comment'] + category_cols].copy()
df_model = df_model.rename(columns={'comment': 'text'})

# Приведение меток к int
df_model[category_cols] = df_model[category_cols].astype(int)



# Подсчёт весов для каждого класса: neg/pos (количество относящихся комментариев к классу и нет)
all_labels = df_model[category_cols].values
pos_counts = all_labels.sum(axis=0)  # Количество положительных примеров для каждого класса
neg_counts = all_labels.shape[0] - pos_counts  # Количество отрицательных примеров для каждого класса
pos_weight = torch.tensor(neg_counts / pos_counts, dtype=torch.float32)

# Вывод результатов
print("Распределение классов:")
print(f"Всего комментариев: {all_labels.shape[0]}\n")

for i, col in enumerate(category_cols):
    print(f"Класс '{col}':")
    print(f"  Положительных: {pos_counts[i]} ({pos_counts[i]/all_labels.shape[0]:.1%})")
    print(f"  Отрицательных: {neg_counts[i]} ({neg_counts[i]/all_labels.shape[0]:.1%})")
    print(f"  Вес класса (neg/pos): {pos_weight[i]:.2f}\n")

print("\nИтоговые веса для loss-функции:")
print(pos_weight)

In [ ]:
from datasets import Dataset

# Создаём датасет HuggingFace
dataset = Dataset.from_pandas(df_model)
dataset = dataset.shuffle(seed=SEED)

# Добавляем поле 'labels' как список значений категорий
def format_labels(example):
    example["labels"] = [float(example[col]) for col in category_cols]
    return example

dataset = dataset.map(format_labels)

# Удаляем отдельные метки (оставляем только 'text' и 'labels')
dataset = dataset.remove_columns(category_cols)

### Токенизация

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
    )

# Токенизируем
tokenized_dataset = dataset.map(tokenize_function, batched=True)

### Подготовка данных для обучения и стратифицированное разбиение

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from datasets import Dataset

# === Стратифицированное разбиение по мультиразметке ===
X = df_model['text'].values
y = df_model[category_cols].values

mskf = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=SEED)
splits = list(mskf.split(X, y))

# Первый сплит: train+val и test
train_val_idx, test_idx = splits[0]
train_val_df = df_model.iloc[train_val_idx].reset_index(drop=True)
test_df = df_model.iloc[test_idx].reset_index(drop=True)

# Второй сплит внутри train_val: train и val
inner_mskf = MultilabelStratifiedKFold(n_splits=4, shuffle=True, random_state=SEED)
inner_splits = list(inner_mskf.split(train_val_df['text'].values, train_val_df[category_cols].values))

train_idx, val_idx = inner_splits[0]
train_df = train_val_df.iloc[train_idx].reset_index(drop=True)
val_df = train_val_df.iloc[val_idx].reset_index(drop=True)

# === Преобразуем pandas -> Hugging Face Dataset ===
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# === Добавляем поле labels ===
def format_labels(example):
    example["labels"] = [float(example[col]) for col in category_cols]
    return example

train_dataset = train_dataset.map(format_labels)
val_dataset = val_dataset.map(format_labels)
test_dataset = test_dataset.map(format_labels)

# Удаляем категориальные метки (оставим только labels)
train_dataset = train_dataset.remove_columns(category_cols)
val_dataset = val_dataset.remove_columns(category_cols)
test_dataset = test_dataset.remove_columns(category_cols)

# === Токенизация ===
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Удалим лишние столбцы (оставим 'text' только в test_dataset)
for ds_name, ds in zip(
    ["train", "val", "test"], [train_dataset, val_dataset, test_dataset]
):
    if ds_name != "test" and "text" in ds.column_names:
        ds = ds.remove_columns(["text"])
    if "_index_level_0_" in ds.column_names:
        ds = ds.remove_columns(["_index_level_0_"])
    if ds_name == "train":
        train_dataset = ds
    elif ds_name == "val":
        val_dataset = ds
    else:
        test_dataset = ds

print("Порядок классов в labels:", category_cols)


### Настройка модели и функции потерь

In [ ]:
import torch.nn.functional as F

class WeightedFocalLoss(torch.nn.Module):
    def __init__(self, pos_weight=None, gamma=2.0):
        super().__init__()
        self.register_buffer('pos_weight', pos_weight)
        self.gamma = gamma

    def forward(self, inputs, targets):
        # Убедимся, что pos_weight на том же устройстве
        pos_weight = self.pos_weight.to(inputs.device)

        bce_loss = F.binary_cross_entropy_with_logits(
            inputs, targets, weight=pos_weight, reduction='none'
        )
        pt = torch.exp(-bce_loss)
        focal_loss = (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()

In [ ]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def __init__(self, loss_fn=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fn = loss_fn

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Удаляем num_items_in_batch, если он есть
        inputs.pop('num_items_in_batch', None)

        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        if self.loss_fn is not None:
            loss = self.loss_fn(logits, labels)
        else:
            loss = outputs.loss

        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

config = AutoConfig.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification",
    # hidden_dropout_prob=0.3,       # Dropout между слоями
    # attention_probs_dropout_prob=0.2  # Dropout в attention
)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config,
    ignore_mismatched_sizes=True
)

loss_fn = WeightedFocalLoss(pos_weight=pos_weight)

### Обучение модели и кросс-валидация

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, roc_curve,
    precision_score, recall_score, auc
)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import os
from copy import deepcopy
from datasets import Dataset
from IPython.display import display

# Отключаем wandb
os.environ["WANDB_DISABLED"] = "true"

# === Метрики ===
def compute_metrics(p):
    preds = p.predictions
    labels = p.label_ids
    binary_preds = (preds > 0.5).astype(int)
    try:
        roc_auc = roc_auc_score(labels, preds, average='macro')
    except ValueError:
        roc_auc = float('nan')
    acc = accuracy_score(labels, binary_preds)
    f1 = f1_score(labels, binary_preds, average='macro')
    return {"accuracy": acc, "f1_macro": f1, "roc_auc_macro": roc_auc}

# === Аргументы обучения ===
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    logging_dir=LOGGING_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=SAVE_TOTAL_LIMIT,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    logging_steps=LOGGING_STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc_macro",
    greater_is_better=True,
    seed=SEED,
    fp16=True,
)

# === K-Fold обучение ===
all_fold_metrics = []
for fold, (train_idx, val_idx) in enumerate(mskf.split(X, y)):
    print(f"\n=== Fold {fold + 1} ===")
    train_df = df_model.iloc[train_idx].reset_index(drop=True)
    val_df = df_model.iloc[val_idx].reset_index(drop=True)

    train_dataset = Dataset.from_pandas(train_df).map(format_labels)
    val_dataset = Dataset.from_pandas(val_df).map(format_labels)
    train_dataset = train_dataset.remove_columns(category_cols).map(tokenize_function, batched=True)
    val_dataset = val_dataset.remove_columns(category_cols).map(tokenize_function, batched=True)

    trainer = CustomTrainer(
        model=deepcopy(model),
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        loss_fn=loss_fn,
        callbacks=[early_stopping]
    )

    trainer.train()

    val_logits = trainer.predict(val_dataset).predictions
    val_probs = torch.sigmoid(torch.tensor(val_logits)).numpy()
    binary_preds = (val_probs > 0.5).astype(int)

    f1 = f1_score(val_df[category_cols].values, binary_preds, average='macro')
    print(f"Fold {fold + 1} F1: {f1:.4f}")
    all_fold_metrics.append(f1)

print(f"\nСредний F1 по 5 фолдам: {np.mean(all_fold_metrics):.4f}")

# === Оптимизация порогов ===
val_logits = trainer.predict(val_dataset).predictions
val_probs = torch.sigmoid(torch.tensor(val_logits)).numpy()

thresholds = []
for i in range(len(category_cols)):
    best_f1, best_t = 0, 0.5
    for t in np.linspace(0.1, 0.9, 81):
        preds = (val_probs[:, i] > t).astype(int)
        f1 = f1_score(val_df[category_cols[i]].values, preds)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    thresholds.append(best_t)

print("Лучшие пороги для каждого класса:", thresholds)

# === Предсказания на тесте ===
preds_output = trainer.predict(test_dataset)
true_labels = preds_output.label_ids
probs = torch.sigmoid(torch.tensor(preds_output.predictions)).numpy()

final_preds = np.zeros_like(probs)
for i, t in enumerate(thresholds):
    final_preds[:, i] = (probs[:, i] > t).astype(int)

# === Финальные метрики ===
def compute_final_metrics(true_labels, final_preds, probs):
    return {
        "accuracy": accuracy_score(true_labels, final_preds),
        "f1_macro": f1_score(true_labels, final_preds, average='macro'),
        "roc_auc_macro": roc_auc_score(true_labels, probs, average='macro'),
    }

metrics = compute_final_metrics(true_labels, final_preds, probs)
print("\n=== Итоговые метрики по порогам ===")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# === Таблица предсказаний ===
probs_df = pd.DataFrame(probs, columns=[col + '_prob' for col in category_cols])
preds_df = pd.DataFrame(final_preds, columns=[col + '_pred' for col in category_cols])
true_df = pd.DataFrame(true_labels, columns=[col + '_true' for col in category_cols])
result_df = pd.concat([probs_df, preds_df, true_df], axis=1)

print("\n=== Таблица предсказаний (первые 10 строк) ===")
styled_table = (result_df.head(10)
                .style.format(precision=2)
                .set_properties({'text-align': 'center'})
                .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]))
display(styled_table)

result_df.to_excel("test_predictions.xlsx", index=False)

# === ROC AUC кривые ===
def compute_roc_auc(true_labels, predicted_probs, label_columns):
    fpr, tpr, roc_auc = {}, {}, {}
    plt.figure(figsize=(12, 8))
    line_styles = ['-', '-', '-', '-']
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for i, label in enumerate(label_columns):
        fpr[i], tpr[i], _ = roc_curve(true_labels[:, i], predicted_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], linestyle=line_styles[i % len(line_styles)],
                 color=colors[i % len(colors)], lw=2,
                 label=f'{label} (AUC = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=1, alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.show()
    return roc_auc

roc_auc = compute_roc_auc(true_labels, probs, category_cols)
avg_roc_auc = np.mean(list(roc_auc.values()))
print(f'\nСредний ROC AUC: {avg_roc_auc:.2f}')